In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, fftfreq
import scipy.signal as signal
import imageio
import sys
from scipy.signal import butter, lfilter, get_window, medfilt
import pywt
import seaborn as sns
from scipy.stats import spearmanr, pearsonr
from sklearn.decomposition import PCA
import statsmodels.graphics.tsaplots as tsaplots
import random
import scipy.stats
import torch
import torch.nn as nn
from scipy.optimize import minimize


import sys
from scipy.signal import savgol_filter, medfilt, find_peaks
from functools import partial

sys.path.append("../")
sys.path.append("../scripts")

from file_io import read_files

In [5]:
%load_ext autoreload
%autoreload 2


In [6]:
gt_df = pd.read_csv("../data/raw/train_labels.csv")
target_star_list = gt_df.planet_id

In [7]:
# Required functions
def smooth_data(data, window_size):
    return savgol_filter(data, window_size, 3)  # window size 51, polynomial order 3


def optimize_breakpoint(
    data, initial_breakpoint=50, window_size=20, buffer_size=5, smooth_window=5
):
    """
    input: 1D time series; first breakpoint guess
    output: first breakpoint
    """
    best_breakpoint = initial_breakpoint
    best_score = float("-inf")
    midpoint = len(data) // 2
    smoothed_data = smooth_data(data, smooth_window)

    data_len = len(data)
    for i in range(-window_size, window_size):
        new_breakpoint = initial_breakpoint + i
        if new_breakpoint - buffer_size < 0 or new_breakpoint + buffer_size >= data_len:
            continue

        region1 = data[: new_breakpoint - buffer_size]
        region2 = data[
            new_breakpoint + buffer_size : data_len - new_breakpoint - buffer_size
        ]
        region3 = data[data_len - new_breakpoint + buffer_size :]

        if len(region1) == 0 or len(region2) == 0 or len(region3) == 0:
            continue

        # calc on smoothed data
        breakpoint_region1 = smoothed_data[
            new_breakpoint - buffer_size : new_breakpoint + buffer_size
        ]
        breakpoint_region2 = smoothed_data[
            -(new_breakpoint + buffer_size) : -(new_breakpoint - buffer_size)
        ]

        mean_diff = abs(np.mean(region1) - np.mean(region2)) + abs(
            np.mean(region2) - np.mean(region3)
        )
        var_sum = np.var(region1) + np.var(region2) + np.var(region3)

        range_at_breakpoint1 = np.ptp(
            breakpoint_region1
        )  # ptp: peak to peak (max - min)
        range_at_breakpoint2 = np.ptp(breakpoint_region2)

        mean_range_at_breakpoint = (range_at_breakpoint1 + range_at_breakpoint2) / 2

        score = mean_diff - 0.5 * var_sum + mean_range_at_breakpoint

        if score > best_score:
            best_score = score
            best_breakpoint = new_breakpoint

    return best_breakpoint, data_len - best_breakpoint


def try_s(signal, p1, p2, deg, s):
    out = np.concatenate((np.arange(p1 - 10), np.arange(p2 + 10, signal.shape[0])))
    x = np.concatenate((out, np.arange(p1, p2)))
    y = np.concatenate((signal[out], signal[p1:p2] * (1 / (1 - s[0]))))

    z = np.polyfit(x, y, deg)
    p = np.poly1d(z)
    q = np.sqrt(np.mean((p(x) - y) ** 2))

    if s < 1e-4:
        return q + 1e3

    return q


def calibrate_signal(signal, ingress, egress, full_output=False):
    """
    input: 1D time series
    output: avg absorption (scalar), time stamp, raw time series, predicted time series
    """

    p1, p2 = ingress, egress

    best_deg, best_score, best_s = 1, 1e12, None
    out = np.concatenate((np.arange(p1 - 10), np.arange(p2 + 10, signal.shape[0])))
    x_out = out
    x_in = np.arange(p1, p2)
    x = np.concatenate((x_out, x_in))

    for deg in range(1, 6):
        f = partial(try_s, signal, p1, p2, deg)
        r = minimize(f, [0.001], method="Nelder-Mead")
        s = r.x[0]

        y = np.concatenate((signal[out], signal[p1:p2] * (1 / (1 - s))))

        z = np.polyfit(x, y, deg)
        p = np.poly1d(z)
        q = np.sqrt(np.mean((p(x) - y) ** 2))

        if q < best_score:
            best_score = q
            best_deg = deg
            best_s = s

    # x = np.concatenate((x_out, x_in))
    y = np.concatenate((signal[out], signal[p1:p2] * (1 / (1 - best_s))))
    z = np.polyfit(x, y, best_deg)
    p = np.poly1d(z)

    if full_output:
        return best_s, x, y, p(x), p
    else:
        return signal - p(np.arange(signal.shape[0])) + signal[0]


def ratio_from_timeseries(signal, phase_0, phase_1, buffer=5, est_mean=False):
    """
    input: 1D time series; phase_0 (start of eclipse); phase_1 (end of eclipse)
    output: predicted absorption ratio

    """

    # out of transit average flux intensiy
    oot_flux = np.concat((signal[: phase_0 - buffer], signal[phase_1 + buffer :]))
    it_flux = signal[phase_0 + buffer : phase_1 - buffer]

    # avg intensity
    oot_avg = np.mean(oot_flux)
    it_avg = np.mean(it_flux)

    # estimated noise
    oot_var = np.std(oot_flux)
    it_var = np.std(it_flux)

    # calculate weighing
    oot_weight = 1 / np.max((oot_var, 1e-5))
    it_weight = 1 / np.max((it_var, 1e-5))
    obs_weight = np.mean((oot_weight, it_weight))

    # if np.isnan(oot_weight).sum() + np.isnan(it_weight).sum() > 0:
    # print("error", oot_weight, it_weight, it_flux, phase_0, phase_1, signal[])
    # print("ok")
    obs_ratio = np.clip((oot_avg - it_avg) / (it_avg + 1e-6), 0, None)

    # print(oot_avg, it_avg, obs_ratio)
    if est_mean:
        est_ratio = obs_weight * obs_ratio + (1 - obs_weight) * est_mean
        return est_ratio

    else:
        est_ratio = obs_ratio
        return est_ratio


def calculate_spectrum(lc, wc, lc_phases, wc_phases):
    """
    input:
    -- lc: obs * t * freq
    -- wc: obs * t
    -- lc_phases: obs * freq * 2 (1 for start 1 for end of eclipse)
    -- wc_phases: obs * 2 (1 for start 1 for end of eclipse)

    output
    -- lc_ratio: obs * freq
    -- wc_ratio: obs * freq (flat line across all freq; this shows the avg pred)
    """

    lc_ratio = np.zeros((lc.shape[0], lc.shape[2]))
    wc_ratio = np.zeros((wc.shape[0], lc.shape[2]))

    for i in range(len(lc)):

        for f in range(lc.shape[2]):
            freq_timeseries = lc[i, :, f]
            freq_ratio = ratio_from_timeseries(
                freq_timeseries, lc_phases[i][f][0], lc_phases[i][f][1]
            )
            lc_ratio[i, f] = freq_ratio

        wc_ratio[i] = ratio_from_timeseries(wc[i], wc_phases[i][0], wc_phases[i][1])

    return lc_ratio, wc_ratio


def subtract_background(timeseries, edge_pixel=5):
    """
    input: observations * t * freq * spatial
    output: observations * t * freq * spatial

    """
    # compute average of the edge pixel, treat them as background noise
    first_5_rows = timeseries[:, :, :, :edge_pixel]
    last_5_rows = timeseries[:, :, :, -edge_pixel:]

    background_noise = (
        np.mean(first_5_rows, axis=3) + np.mean(last_5_rows, axis=3)
    ) / 2

    print(background_noise.shape)

    # Step 3: Subtract the background noise from all rows along axis 2
    # Use broadcasting to subtract it from each row of the 3rd axis
    data_corrected = timeseries - background_noise[:, :, :, np.newaxis]

    return data_corrected


def spatial_integration(timeseries, range_start=8, range_end=24):
    """
    input: observation * t * freq * spatial
    output: observation * t * freq

    """

    timeseries = timeseries[:, :, :, range_start:range_end].sum(axis=3)

    return timeseries


def bin_frequencies(time_series, k):
    """
    Bin the frequencies of a time series such that each bin has roughly equal intensity.

    Parameters:
    - time_series: numpy.ndarray, the time series data of shape (observations * time_steps, frequency).
    - k: int, the number of bins.

    Returns:
    - binned_series: numpy.ndarray, the binned time series data of shape (time_steps, k).
    """

    full_binned_series = np.zeros_like(time_series)
    # full_binned_series = np.zeros((time_series.shape[0], time_series.shape[1], k))

    for i in range(time_series.shape[0]):
        # Compute the total intensity for each frequency
        total_intensity = np.sum(time_series[i], axis=0)

        # Compute the cumulative intensity
        cumulative_intensity = np.cumsum(total_intensity)

        # Determine the total intensity and the target intensity per bin
        total_intensity_sum = cumulative_intensity[-1]
        target_intensity_per_bin = total_intensity_sum / k

        # Initialize the binned series
        time_steps, freq = time_series[i].shape
        binned_series = np.zeros((time_steps, k))

        # Bin the frequencies
        bin_idx = 0
        bin_intensity = 0
        bin_indices = [0]
        for j in range(freq):
            if (
                bin_intensity + total_intensity[j] > target_intensity_per_bin
                and bin_idx < k - 1
            ):
                bin_indices.append(j)
                bin_idx += 1
                bin_intensity = 0
            binned_series[:, bin_idx] += time_series[i, :, j]
            bin_intensity += total_intensity[j]
        bin_indices.append(j + 1)
        # print(bin_indices)
        # print(binned_series.shape)
        for bin_index in range(len(bin_indices) - 1):
            full_binned_series[i][
                :, bin_indices[bin_index] : bin_indices[bin_index + 1]
            ] = binned_series[:, bin_index].reshape(-1, 1)
    return full_binned_series


def detector_runner(timeseries, detector_method):
    """
    input: # obs * t * freq
    output:
        -- LC phases: # obs * freq * 2
        -- WC phases: # obs * 2
    """

    lc_phases = np.zeros((timeseries.shape[0], timeseries.shape[2], 2), dtype=int)
    wc_phases = np.zeros((timeseries.shape[0], 2), dtype=int)

    # wc_timeseries = timeseries.sum(axis=2)

    for i, observation in enumerate(timeseries):
        for freq in range(len(observation[0])):
            lc_phases[i, freq, 0], lc_phases[i, freq, 1] = detector_method(
                observation[:, freq]
            )
        wc_phases[i, 0], wc_phases[i, 1] = detector_method(observation.sum(axis=(1)))

    return lc_phases, wc_phases


def model_runner(timeseries, lc_phases, wc_phases, method):
    """
    input:
    -- timeseries: obs * t * freq
    -- lc_phases: obs * freq * 2
    -- wc_phases: obs * 2

    output:
    -- fitted_lc: obs * t * freq
    -- fitted_wc: obs * t
    -- lc_flux_ratio: obs * freq
    -- wc_flux_ratio: obs
    """

    fitted_lc = np.zeros(timeseries.shape, dtype=int)
    fitted_wc = np.zeros((timeseries.shape[0], timeseries.shape[1]), dtype=int)

    for i, observation in enumerate(timeseries):
        for j in range(timeseries.shape[2]):
            lc_phase_1 = lc_phases[i, j, 0]
            lc_phase_2 = lc_phases[i, j, 1]

            # print("testing", lc_phase_1, lc_phase_2)
            # print(timeseries[i, :, j])
            fitted_lc[i, :, j] = method(
                timeseries[i, :, j], lc_phase_1 + 5, lc_phase_2 - 5
            )  # include buffer here
        wc_phase_1 = wc_phases[i, 0] + 5  # include buffer here
        wc_phase_2 = wc_phases[i, 1] - 5  # include buffer here

        fitted_wc[i] = method(
            signal.medfilt(timeseries[i].sum(axis=1), 5), wc_phase_1, wc_phase_2
        )

    return fitted_lc, fitted_wc

In [1]:
# Prediction sequence
# selected_targets = target_star_list[:2]


def prediction_logic():

    # read files
    airs_selected = []
    fgs_selected = []
    # gt_selected = []

    # for target in target_list:
    #     # airs, fgs, gt = read_files(target, gt_df)
    #     airs = np.load(f"../data/processed/{target_star}_airs.npy")
    #     fgs = np.load(f"../data/processed/{target_star}_fgs1.npy")
    #     airs_selected.append(airs)
    #     fgs_selected.append(fgs)
    #     # gt_selected.append(gt)

    airs_selected = np.load("airs_v4.npy")
    # fgs_selected = np.vstack(fgs_selected)
    # gt_selected = np.vstack(gt_selected)
    print("file read")

    # PREPROCESS
    # 1. subtracting the estimated background signal;
    airs_selected = subtract_background(airs_selected)
    # 2. integrate spatially across the center of the detector. Output shape of observations * t * freq
    airs_selected = spatial_integration(airs_selected)

    # 3. additional smoothing step
    # airs_selected = wavelet_smoothing(airs_selected, 5) # wavelet smoothing seems to decrease the max-min range of signal, we will skip this for now

    # DE-TREND
    # phase detection - returns the estimated instants at which entry and exit occurs for each eclipse event
    lc_phases, wc_phases = detector_runner(airs_selected, optimize_breakpoint)

    lc_phases = np.tile(
        wc_phases[:, np.newaxis, :], (1, 282, 1)
    )  # we will use wc_phases for both lc and wc for the time being, due to resolution limits

    # Frequency Binning:
    binned_airs_selected = bin_frequencies(airs_selected, 10)

    fitted_lc, fitted_wc = model_runner(
        binned_airs_selected, lc_phases, wc_phases, calibrate_signal
    )  # added median smoothing to the model runner

    # Cal prediction
    lc_ratio, wc_ratio = calculate_spectrum(fitted_lc, fitted_wc, lc_phases, wc_phases)

    return (
        airs_selected,
        lc_phases,
        wc_phases,
        fitted_lc,
        fitted_wc,
        # gt_selected,
        lc_ratio,
        wc_ratio,
    )
    # print(airs_selected.shape)


(
    airs_selected,
    lc_phases,
    wc_phases,
    fitted_lc,
    fitted_wc,
    # gt_selected,
    lc_ratio,
    wc_ratio,
) = prediction_logic()

NameError: name 'target_star_list' is not defined

In [1025]:
sample_sub = pd.read_csv("../data/raw/sample_submission.csv")

In [1154]:
gt_df = pd.read_csv("../data/raw/train_labels.csv")
del gt_df["planet_id"]

In [1165]:
preds = lc_ratio[:, ::-1]
constant_value = np.full((preds.shape[0], 1), lc_ratio.mean())
preds = np.hstack((constant_value, preds))

sigmas = np.vstack(
    [np.std((lc_ratio[:, ::-1] - gt_df.iloc[:, 1:].values), axis=0) * i]
    * lc_ratio.shape[0]
)
constant_value = np.full((sigmas.shape[0], 1), sigmas.mean())
sigmas = np.hstack((constant_value, sigmas))

sub_frame = pd.DataFrame(
    np.concatenate([preds, sigmas], axis=1), columns=sample_sub.columns[1:]
)

1.0 0.3938202007381944
1.01 0.39445602621175563
1.02 0.395039057252732
1.03 0.3955716660641241
1.04 0.396056104968149
1.05 0.3964945134061458
1.06 0.3968889244715333
1.07 0.3972412710108922
1.08 0.3975533913253136
1.09 0.39782703450150475
1.1 0.39806386539970634
1.11 0.3982654693233
1.12 0.39843335639296773
1.1300000000000001 0.39856896564644734
1.1400000000000001 0.3986736688832508
1.1500000000000001 0.39874877427221167
1.1600000000000001 0.39879552973831683
1.1700000000000002 0.3988151261440305
1.1800000000000002 0.39880870027912574
1.1900000000000002 0.3987773376720115
1.2000000000000002 0.3987220752345316
1.2100000000000002 0.398643903751339
1.2200000000000002 0.3985437702241226
1.2300000000000002 0.39842258008020054
1.2400000000000002 0.39828119925430905
1.2500000000000002 0.3981204561517799
1.2600000000000002 0.3979411435006977
1.2700000000000002 0.3977440201001011
1.2800000000000002 0.3975298124707928
1.2900000000000003 0.39729921641484883
1.3000000000000003 0.3970528984895105
1